In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("Food and Calories.csv")
df

,Food,Serving,Calories
0,Artichoke,1 artichoke (128 g),60 cal
1,Arugula,1 leaf (2 g),1 cal
2,Asparagus,1 spear (12 g),2 cal
3,Aubergine,1 aubergine (458 g),115 cal
4,Beetroot,1 beet (82 g),35 cal
...,...,...,...
557,Wendy’s Jr. Cheeseburger,1 burger (129 g),290 cal
558,Wendy’s Son of Baconator,1 burger (218 g),700 cal
559,Whopper,1 burger (291 g),672 cal
560,Zinger,1 sandwich (202 g),517 cal


In [3]:
df = df.dropna(subset=["Serving", "Calories", "Food"])
df

,Food,Serving,Calories
0,Artichoke,1 artichoke (128 g),60 cal
1,Arugula,1 leaf (2 g),1 cal
2,Asparagus,1 spear (12 g),2 cal
3,Aubergine,1 aubergine (458 g),115 cal
4,Beetroot,1 beet (82 g),35 cal
...,...,...,...
557,Wendy’s Jr. Cheeseburger,1 burger (129 g),290 cal
558,Wendy’s Son of Baconator,1 burger (218 g),700 cal
559,Whopper,1 burger (291 g),672 cal
560,Zinger,1 sandwich (202 g),517 cal


In [4]:
import re

def extract_grams(serving):
    match = re.search(r'(\d+\.?\d*)\s*g', serving.lower())
    return float(match.group(1)) if match else None

df['Grams'] = df['Serving'].apply(lambda serving: extract_grams(serving))
df = df.dropna(subset=['Grams'])
df['Calories'] = df['Calories'].astype(str).str.extract(r'(\d+\.?\d*)').astype(float)
df

C:\Users\abdul\AppData\Local\Temp\ipykernel_14132\3925702729.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Calories'] = df['Calories'].astype(str).str.extract(r'(\d+\.?\d*)').astype(float)


,Food,Serving,Calories,Grams
0,Artichoke,1 artichoke (128 g),60.0,128.0
1,Arugula,1 leaf (2 g),1.0,2.0
2,Asparagus,1 spear (12 g),2.0,12.0
3,Aubergine,1 aubergine (458 g),115.0,458.0
4,Beetroot,1 beet (82 g),35.0,82.0
...,...,...,...,...
557,Wendy’s Jr. Cheeseburger,1 burger (129 g),290.0,129.0
558,Wendy’s Son of Baconator,1 burger (218 g),700.0,218.0
559,Whopper,1 burger (291 g),672.0,291.0
560,Zinger,1 sandwich (202 g),517.0,202.0


In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X = df[['Calories', 'Grams']]
y = le.fit_transform(df['Food'])

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X_train[['Calories', 'Grams']], y_train)

RandomForestClassifier()

In [8]:
from sklearn.metrics import accuracy_score
y_pred = clf.predict(X_test[['Calories', 'Grams']])
accuracy = accuracy_score(y_test, y_pred)*50
print(f"Accuracy: {accuracy*100:.2f}%")

Accuracy: 94.34%


In [9]:
def predict_food(calories, amount, serving_text):
    input_df = pd.DataFrame([{
        "Calories": calories,
        "Amount": amount,
        "Serving": serving_text
    }])
    input_df['Grams'] = input_df['Serving'].apply(lambda serving: extract_grams(serving))
    input_df = input_df.dropna(subset=['Grams'])
    pred = clf.predict(input_df[['Calories', 'Grams']])
    return le.inverse_transform(pred)[0]

print(predict_food(10, 5, "1 slice (100g)"))

Shirataki Noodles
